Yahoo Data Scraping

In [ ]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\noten\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
%pip install bs4
%pip install requests


Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import requests

In [5]:
url = "https://www.investing.com/equities/amazon-com-inc"
page = requests.get(url)

print(page.status_code) # 200 means successful request

200


In [6]:
soup = BeautifulSoup(page.text, 'html.parser')

In [7]:
# Some useful CSS class tags for webscraping

h1_class = "mb-2.5 text-left text-xl font-bold leading-7 text-[#232526] md:mb-2 md:text-3xl md:leading-8 rtl:soft-ltr"
price_class = "text-5xl/9 font-bold text-[#232526] md:text-[42px] md:leading-[60px]"
price_change_class = "instrument-price-change"
percent_change_class = "instrument-price-change-percent"

change_classes = ["flex items-center gap-2 text-base/6 font-bold md:text-xl/7 rtl:force-ltr text-negative-main", 
                  "flex items-center gap-2 text-base/6 font-bold md:text-xl/7 rtl:force-ltr text-positive-main"]

In [8]:
"""
Beautiful Soup returns a parsed tree, so we can now navigate the tree and pick the element we want, even though we don't have the exact CSS class.
What we do here is go up in the heirarchy and find a parent div we can exploit. Then, we can use find_all('span') to make a list of all the elements
containing the span tag - which we know our target data uses. And because it's a list, we can now easily navigate it and pick those we need.  

soup.find(html tag, class = ...) returns the first occurence of a class matching the specified one with the matching html tag.
"""

company = soup.find('h1', {'class': h1_class}).text
price = soup.find('div', {'class': price_class}).text

loss_test = soup.find('div', {'class': change_classes[0]})
if loss_test == None:
    price_change = soup.find('div', {'class': change_classes[1]}).find_all('span')[0].text
    percent_change = soup.find('div', {'class': change_classes[1]}).find_all('span')[1].text[1:-2] # remove unnecessary brackets and percent    
else:    
    price_change = loss_test.find_all('span')[0].text
    percent_change = loss_test.find_all('span')[1].text[1:-2] # remove unnecessary brackets and percent

# print(soup.find('div', {'class': change_class}).find_all('span')[1].text[1:-2])

In [9]:
# test run:
print('Loading ' + url)
print(company, price, price_change, percent_change) # only gets one value - can we get more?

Loading https://www.investing.com/equities/amazon-com-inc
Amazon.com Inc (AMZN) 179.62 +5.95 +3.43


In [10]:
# # https://www.investing.com/equities/amazon-com-inc-historical-data - fill in data from then (1/1/2018) to now (whatever today is)

period_url = "https://www.investing.com/equities/amazon-com-inc-historical-data"
period_page = requests.get(period_url)
print(period_page.status_code)

200


In [ ]:
period_soup = BeautifulSoup(period_page.text, 'html.parser')
all_date_class = "freeze-column-w-1 w-full overflow-x-auto text-xs leading-4"
# test_class = "relative h-[41px] after:absolute after:bottom-0 after:left-0 after:right-0 after:h-px after:bg-[#ECEDEF] hover:bg-[#F5F5F5] historical-data-v2_price__atUfP"
table = period_soup.find('table', {'class': all_date_class})
rows = table.find_all('tr')

i = 0
dates = []
prices = []
changes = []
for row in rows:
    # cells = row.find_all(['th', 'td'])
    cells = row.find_all('td')
    for cell in cells:
        match i:
            case 0:
                dates.append(cell.text)
            case 1:
                prices.append(cell.text)
            case 6:
                changes.append(cell.text[:-1])
        i += 1
        # print(i)
        if i > 6:
            i = 0
print(dates)
print(prices)
print(changes)            

['04/26/2024', '04/25/2024', '04/24/2024', '04/23/2024', '04/22/2024', '04/19/2024', '04/18/2024', '04/17/2024', '04/16/2024', '04/15/2024', '04/12/2024', '04/11/2024', '04/10/2024', '04/09/2024', '04/08/2024', '04/05/2024', '04/04/2024', '04/03/2024', '04/02/2024', '04/01/2024', '03/28/2024', '03/27/2024', '03/26/2024']
['178.86', '173.66', '176.59', '179.55', '177.23', '174.63', '179.22', '181.28', '183.32', '183.62', '186.13', '189.05', '185.95', '185.67', '185.19', '185.07', '180.00', '182.41', '180.69', '180.97', '180.38', '179.83', '178.30']
['+2.99', '-1.66', '-1.65', '+1.31', '+1.49', '-2.56', '-1.14', '-1.11', '-0.16', '-1.35', '-1.54', '+1.67', '+0.15', '+0.26', '+0.06', '+2.82', '-1.32', '+0.95', '-0.15', '+0.33', '+0.31', '+0.86', '-0.78']
